In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string



In [26]:
fake = pd.read_csv('Fake (1).csv')
true = pd.read_csv('True (1).csv')

In [27]:
fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [28]:
fake['class'] =0
true['class'] =1

In [29]:
data = pd.concat([fake,true], axis= 0)

In [30]:
data.sample(10)

,title,text,subject,date,class
11190,British lawmakers debate banning Trump after M...,LONDON (Reuters) - British lawmakers on Monday...,politicsNews,"January 18, 2016",1
10374,REPORT: NJ GOV Chris Christie’s BIG ‘Mistake’ ...,Chris Christie knows how to create awkward pol...,politics,"Jul 18, 2017",0
19397,France to skip 2018 Winter Games if security n...,PARIS/SEOUL (Reuters) - France s Winter Olympi...,worldnews,"September 21, 2017",1
10243,KID ROCK BAND MEMBER PREDICTS: ‘That F*cker Co...,Kid Rock Keyboardist Jimmie Bones is shocked b...,politics,"Aug 3, 2017",0
12901,MEDIA BUSTED! 2003 VIDEO Of TRUMP Proving His ...,,politics,"Sep 27, 2016",0
10518,Obama says he will stay in Washington after he...,(Reuters) - President Barack Obama said on Thu...,politicsNews,"March 3, 2016",1
10081,Creditors sue to block Puerto Rico development...,SAN JUAN (Reuters) - Creditors of Puerto Rico’...,politicsNews,"April 4, 2016",1
4275,"Mike Pence Cancels ANOTHER Trump Event, Campa...",Donald Trump s lewd comments about sexually as...,News,"October 10, 2016",0
15549,"[VIDEO] OUR DIVIDER IN CHIEF, BARACK OBAMA, IN...",Our 44th President just doing his part to keep...,politics,"Jun 22, 2015",0
19705,YIKES! Former Dem Pollster Makes Bold Predicti...,Jimmy Carter s Democrat pollster Pat Caddell i...,left-news,"Oct 31, 2016",0


In [31]:
data = data.drop(["title", "subject", "date"], axis= 1)

In [32]:
data.reset_index(inplace=True)

In [33]:
data.drop(['index'],axis= 1,inplace=True)

In [46]:
data.sample(5)

,text,class
31660,washington reuters it was classic hillary ...,1
20133,will hillary s health be the only thing she ca...,0
19969,remember will and jada pinkett smith s contrib...,0
37362,buenos aires reuters argentine president m...,1
36997,copenhagen reuters iceland s opposition le...,1


In [35]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\[.*?\]", "", text)  # Fixed regex pattern
    text = re.sub(r"\W", " ", text)  # Removed extra backslash
    text = re.sub(r"https?://\S+|www\.\S+", "", text)  # Fixed URL regex
    text = re.sub(r"<.*?>+", "", text)  # Fixed angle bracket regex
    text = re.sub(r"[%s]" % re.escape(string.punctuation), "", text)  # Removes punctuation
    text = re.sub(r"\n", "", text)  # Removes newlines
    text = re.sub(r"\w*\d\w*", "", text)  # Removes words with numbers
    return text

In [36]:
# Applying the function to the dataset
data["text"] = data["text"].apply(clean_text)

In [37]:
# Splitting the dataset into training and testing sets
x = data["text"]
y = data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)


In [38]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [39]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [40]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9860133630289533

In [41]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [42]:
joblib.dump(vectorizer, "vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']